In [7]:
import sys
print(sys.executable[:-10] + "bin\\tessdata")

E:\Anaconda\envs\Project-Fugu-Manga-Translator\bin\tessdata


In [15]:
import pytesseract
import cv2
import sys
tessdata_dir = sys.executable[:-10] + "bin\\tessdata"
!set TESSDATA_PREFIX={tessdata_dir}

In [16]:
!echo %TESSDATA_PREFIX%

%TESSDATA_PREFIX%


In [2]:
img = cv2.imread("../datasets/Manga109/Manga109_released_2021_12_30/images/AisazuNihaIrarenai/005.jpg")

In [21]:
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [44]:
tess_config = "-l jpn_vert --psm 12"

h, w, c = img.shape
d = pytesseract.image_to_data(img, config=tess_config, output_type=pytesseract.Output.DICT)
print(d)
n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 60:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv2.imshow("test", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("output/ocr-test.jpg", img)

{'level': [1, 2, 3, 4, 5, 5, 2, 3, 4, 5, 5, 5, 5, 2, 3, 4, 5, 5, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 2, 3, 4, 5, 5, 5, 5, 2, 3, 4, 5, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 5, 5, 2, 3, 4, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 5, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 5, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 5, 5, 5, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 2, 3, 4, 5, 5, 5, 5, 2, 3, 4, 5, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 5, 5, 5, 2, 3, 4, 5, 5, 5, 5, 2, 3, 4, 5, 5, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 5, 2, 3, 4, 5, 5, 5, 2, 3, 4, 5, 5, 2, 3,

True

In [18]:
pytesseract.__version__

'0.3.9'

In [19]:
!where tesseract

E:\Anaconda\envs\Project-Fugu-Manga-Translator\Library\bin\tesseract.exe
